# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp (F),Humidity,Cloudiness,Wind Speed,Country,Date
0,0,geraldton,-28.7667,114.6000,70.21,78,75,17.27,AU,1704749064
1,1,baruun-urt,46.6806,113.2792,-11.54,94,24,4.74,MN,1704749066
2,2,port-aux-francais,-49.3500,70.2167,41.54,80,92,27.20,TF,1704749066
3,3,edinburgh of the seven seas,-37.0676,-12.3116,65.25,92,100,29.06,SH,1704749067
4,4,blackmans bay,-43.0167,147.3167,59.13,75,86,3.74,AU,1704749067


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
# Configure the map plot
map_plot1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size="Humidity",
    color="orange",
    line_color="black",
    tiles = "OSM")

# Display the map
map_plot1

/Users/nancyz/anaconda3/envs/dev39/lib/python3.11/site-packages/dask/dataframe/_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [ ]:
# Narrow down cities that fit criteria and drop any results with null values
criteria_df = city_data_df[(city_data_df["Max Temp (F)"] < 80) & (city_data_df["Max Temp (F)"] > 40) &
                        (city_data_df["Wind Speed"] < 6) & (city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
filtered_df = criteria_df.dropna()

# Display sample data
filtered_df

,City_ID,City,Lat,Lng,Max Temp (F),Humidity,Cloudiness,Wind Speed,Country,Date
15,15,bethel,41.3712,-73.4140,41.65,65,0,5.75,US,1704748851
42,42,tura,25.5198,90.2201,56.10,80,0,2.51,IN,1704749075
54,54,bhanpuri,21.1000,80.9167,63.81,62,0,5.57,IN,1704749106
71,71,bhamo,24.2667,97.2333,52.61,71,0,1.41,MM,1704749116
192,192,yanqul,23.5905,56.5501,57.47,56,0,5.35,OM,1704749192
231,231,goundi,9.3627,17.3660,76.60,15,0,3.33,TD,1704749344
273,273,kanina khas,28.3333,76.3167,48.07,65,0,2.51,IN,1704749369
315,315,barpathar,26.3000,93.8667,55.15,84,0,1.12,IN,1704749390
381,381,ceres,37.5949,-120.9577,55.06,50,0,3.44,US,1704749440
396,396,ha'il,27.5219,41.6907,55.42,67,0,4.61,SA,1704749167


### Step 3: Create a new DataFrame called `hotel_df`.

In [ ]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
15,bethel,US,41.3712,-73.4140,65,
42,tura,IN,25.5198,90.2201,80,
54,bhanpuri,IN,21.1000,80.9167,62,
71,bhamo,MM,24.2667,97.2333,71,
192,yanqul,OM,23.5905,56.5501,56,
231,goundi,TD,9.3627,17.3660,15,
273,kanina khas,IN,28.3333,76.3167,65,
315,barpathar,IN,26.3000,93.8667,84,
381,ceres,US,37.5949,-120.9577,50,
396,ha'il,SA,27.5219,41.6907,67,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [ ]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "limit":20,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lng = row["Lng"]
    lat = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
bethel - nearest hotel: Hampton Inn Danbury
tura - nearest hotel: No hotel found
bhanpuri - nearest hotel: पायल ढाबा
bhamo - nearest hotel: Winlight Hotel
yanqul - nearest hotel: No hotel in this town
goundi - nearest hotel: No hotel found
kanina khas - nearest hotel: No hotel found
barpathar - nearest hotel: No hotel found
ceres - nearest hotel: Howard Johnson by Wyndham Modesto Ceres
ha'il - nearest hotel: فندق كلاسيك
shindand - nearest hotel: No hotel found
turaif - nearest hotel: المودة
arar - nearest hotel: دار أوتيل Dar Otel
bam - nearest hotel: هتل جهانگردی بم
nalhati - nearest hotel: No hotel found
mao - nearest hotel: Hevresac


,City,Country,Lat,Lng,Humidity,Hotel Name
15,bethel,US,41.3712,-73.4140,65,Hampton Inn Danbury
42,tura,IN,25.5198,90.2201,80,No hotel found
54,bhanpuri,IN,21.1000,80.9167,62,पायल ढाबा
71,bhamo,MM,24.2667,97.2333,71,Winlight Hotel
192,yanqul,OM,23.5905,56.5501,56,No hotel in this town
231,goundi,TD,9.3627,17.3660,15,No hotel found
273,kanina khas,IN,28.3333,76.3167,65,No hotel found
315,barpathar,IN,26.3000,93.8667,84,No hotel found
381,ceres,US,37.5949,-120.9577,50,Howard Johnson by Wyndham Modesto Ceres
396,ha'il,SA,27.5219,41.6907,67,فندق كلاسيك


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [4]:
%%capture --no-display

# Configure the map plot
map_plot2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    scale = 1,
    color= "City",
    line_color="black",
    tiles = "OSM",
    frame_width = 500,
    frame_height = 400,
    legend = True,
    hover_cols=["Country", "Hotel Name"]
)
# Display the map
map_plot2

NameError: name 'hotel_df' is not defined